In [157]:
from datetime import datetime, timedelta
import json
from pathlib import Path
import pytz
import requests
import time
import pandas as pd

#get token
user_id = 'XXXXXXXXXXXXXXXXXX'
user_secret='XXXXXXXXXXXXXXXXXX'
url = f'https://iiko.biz:9900/api/0/auth/access_token?user_id={user_id}&user_secret={user_id}'
access_token = requests.get(url).text.strip('"')

#part of data for query
organizationId = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
deliveryStatus = 'CLOSED'
request_timeout = '00:05:00'
df_half_year = pd.DataFrame()

#get data for half year
for i in range(0,6):
    dateFrom = (datetime.today()-timedelta(days=30*(i+1)+1+i)).strftime('%Y-%m-%d')
    dateTo = (datetime.today()-timedelta(days=30*i+1+i)).strftime('%Y-%m-%d')
    url = f'https://iiko.biz:9900/api/0/orders/deliveryOrders?access_token={access_token}&organization={organizationId}&dateFrom={dateFrom}&dateTo={dateTo}&deliveryStatus={deliveryStatus}&request_timeout={request_timeout}'
    json_data = json.loads(requests.get(url).text)
    df_half_year = pd.concat([df_half_year, pd.json_normalize(json_data['deliveryOrders'])])


#get type of sourse from comment fild
def find_source(x):
    if x.find('DC') ==-1:
        return 0
    else:
        return 1

#clianing data
actual_colomns = ['actualTime',	'comment', 'customerName',	'customerPhone', 'sum']
df_half_year=df_half_year[actual_colomns]
df_half_year['source'] = df_half_year['comment'].apply(find_source)
df_half_year = df_half_year.drop('comment', axis=1)
df_half_year = df_half_year.rename(columns={"actualTime": "Время открытия", "customerName": "ФИО", "customerPhone": "Телефон", "sum": "Сумма заказа", "source": "Источник"})
df_half_year.reset_index(drop=True, inplace=True)
df_half_year['Время открытия'] = pd.to_datetime(df_half_year['Время открытия'])

grouped_1 = df_half_year.groupby(['Телефон'])['Источник'].agg(['sum', 'count'])
grouped_1 = grouped_1.rename(columns={"sum": "Агрегаторы", "count": "Заказов всего"})
grouped_1 = grouped_1[grouped_1['Заказов всего']!=1]

df_half_year = df_half_year.sort_values(by=['Телефон', 'Время открытия'], ascending = [True, True])
df_half_year['Между заказами'] = df_half_year.groupby(['Телефон'])['Время открытия'].diff().fillna(timedelta(seconds=0))
df_half_year = df_half_year[df_half_year['Между заказами']!=timedelta(seconds=0)]

def mean_group(df):
    return df.sum()/(df.shape[0])

def last_element(df):
    return df.tail(1).iloc[0]

def fio(df):
    return df.unique()
    
grouped_2 = df_half_year.groupby(['Телефон']).agg({'Между заказами': mean_group, 'Время открытия': last_element, 'Сумма заказа': 'mean', 'ФИО': fio})
grouped_2 = grouped_2.rename(columns={"Время открытия": "Крайний заказ", "Сумма заказа": "Средний чек"})

clients = grouped_1.merge(grouped_2, on='Телефон')

clients['Не заказывает'] = (clients['Крайний заказ'].max() - clients['Крайний заказ'])
clients = clients[ (2*clients['Между заказами'] < clients['Не заказывает']) & (2*clients['Между заказами'] + timedelta(days=1) > clients['Не заказывает'])].sort_values(by='Заказов всего', ascending=False)

columns= ['ФИО', 'Заказов всего', 'Агрегаторы', 'Не заказывает',  'Между заказами', 'Крайний заказ', 'Средняя сумма']

clients[columns]